In [12]:
import requests
from bs4 import BeautifulSoup

#third party advice


# 1. Obtener la URL
url = "https://books.toscrape.com/"
respuesta_obtenida = requests.get(url)
html_obtenido = respuesta_obtenida.text  # HTML como string

# 2. Parsear ese HTML con BeautifulSoup
soup = BeautifulSoup(html_obtenido, 'html.parser')  

#categorias

# # --- Método 1: usando ul_menu y for ---
# categorias = soup.find('ul', class_='nav nav-list')
# items = categorias.find_all('li')

# diccionario_links = {}

# for li in items:
#     enlace = li.find('a')
#     texto = enlace.text.strip()
#     link = enlace['href']
#     diccionario_links[texto] = link
    
# print('Método 1:', diccionario_links)

# # --- Método 2: usando comprensión de diccionario y select ---
# diccionario_links2 = {a.text.strip(): a['href'] for a in soup.select('ul.nav.nav-list li a')}
# print('Método 2:', diccionario_links2)

# --- Método 3: usando un bucle for y select, pero en menos líneas ---

diccionario_links3 = {}
for a in soup.select('ul.nav.nav-list li a'):
    diccionario_links3[a.text.strip()] = url + a['href']
print(diccionario_links3)


for link in diccionario_links3.keys():
    link = requests.get(url)
    link_text = link.text
    if link_text.find('ul', class_ = 'pager') == True:
        for page in link



SyntaxError: expected ':' (111805284.py, line 47)

In [ ]:
import requests
from bs4 import BeautifulSoup
libros = None


base_url = "https://books.toscrape.com/catalogue/page-{}.html"


#funcion para recorrer todas las paginas 

for page in range(1, 2):  # Hay 50 páginas en total
    url = base_url.format(page)
    respuesta = requests.get(url)
    soup = BeautifulSoup(respuesta.text, 'html.parser')

    # Aquí haces scraping de los libros de esa página
    libros = soup.find_all('article', class_='product_pod')
    print(f"📄 Página {page} -> {len(libros)} libros encontrados")

# recorrer_paginas()


#   

for libro in libros:
    titulo = libro.h3.a['title']
    precio = libro.find('p', class_='price_color').text
    disponibilidad = libro.find('p', class_='instock availability').text.strip()

    # Enlace al detalle del libro (relativo -> convertir a absoluto)
    enlace_relativo = libro.h3.a['href']
    enlace_absoluto = "https://books.toscrape.com/catalogue/" + enlace_relativo

    print(f"📚 {titulo} | 💵 {precio} | 📦 {disponibilidad} | 🔗 {enlace_absoluto}")
    
    
    
detalle = requests.get(enlace_absoluto)
soup_detalle = BeautifulSoup(detalle.text, 'html.parser')

# Categoría: está en un breadcrumb (<ul class="breadcrumb">)
categoria = soup_detalle.find('ul', class_='breadcrumb').find_all('a')[-1].text

print(f"📚 {titulo} | 💵 {precio} | 📦 {disponibilidad} | 📂 {categoria}")

    
    





📄 Página 1 -> 20 libros encontrados
📚 A Light in the Attic | 💵 Â£51.77 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html
📚 Tipping the Velvet | 💵 Â£53.74 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html
📚 Soumission | 💵 Â£50.10 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/soumission_998/index.html
📚 Sharp Objects | 💵 Â£47.82 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/sharp-objects_997/index.html
📚 Sapiens: A Brief History of Humankind | 💵 Â£54.23 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html
📚 The Requiem Red | 💵 Â£22.65 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/the-requiem-red_995/index.html
📚 The Dirty Little Secrets of Getting Your Dream Job | 💵 Â£33.34 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html
📚 The Coming Woman: A Novel Based on the Li

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}


# Definimos la URL base del sitio web a scrapear
url = "https://books.toscrape.com/"

# Hacemos una petición GET a la página principal para obtener el HTML
respuesta = requests.get(url,headers=headers, timeout=30)
soup = BeautifulSoup(respuesta.text, 'html.parser')

# Obtenemos todas las categorías disponibles (excepto 'All books') y armamos un diccionario con el nombre y el link absoluto
categorias = {
    a.text.strip(): url + a['href'] 
    for a in soup.select('ul.nav.nav-list li a') 
    if a['href'] != 'catalogue/category/books_1/index.html'
}  # Excluye 'All books' de la lista de categorías

print(f"Categorías encontradas: {len(categorias)}")
print(categorias)

# Creamos una lista vacía donde guardaremos la información de todos los libros
libros_data = []

# Recorremos cada categoría encontrada
for nombre_cat, url_cat in categorias.items():
    page = 1  # Empezamos por la primera página de la categoría
    while True:
        # Armamos la URL de la página actual de la categoría
        if page == 1:
            url_pagina = url_cat
        else:
            url_pagina = url_cat.replace('index.html', f'page-{page}.html')

        # Hacemos la petición GET a la página de la categoría
        resp_cat = requests.get(url_pagina, timeout = 30)
        soup_cat = BeautifulSoup(resp_cat.text, 'html.parser')

        # Buscamos todos los libros en la página actual
        libros = soup_cat.find_all('article', class_='product_pod')

        # Si no hay libros, significa que no hay más páginas en esta categoría
        if not libros:
            break

        # Recorremos cada libro encontrado en la página
        for libro in libros:
            # Extraemos el título del libro
            titulo = libro.h3.a['title']
            # Extraemos el precio
            precio = libro.find('p', class_='price_color').text
            # Extraemos la disponibilidad (stock)
            disponibilidad = libro.find('p', class_='instock availability').text.strip()
            # Extraemos el rating (estrellas) como texto (ej: 'Three')
            rating = libro.p['class'][1] if len(libro.p['class']) > 1 else 'None'
            # Obtenemos el enlace relativo al detalle del libro
            enlace_rel = libro.h3.a['href']
            # Convertimos el enlace relativo a absoluto
            if not enlace_rel.startswith('http'):
                if enlace_rel.startswith('../'):
                    enlace_rel = enlace_rel.replace('../', '')
                enlace_abs = url + 'catalogue/' + enlace_rel
            else:
                enlace_abs = enlace_rel

            # Hacemos una petición GET al detalle del libro para obtener más información
            detalle = requests.get(enlace_abs, timeout =30)
            soup_detalle = BeautifulSoup(detalle.text, 'html.parser')

            # Intentamos extraer la categoría desde el breadcrumb del detalle (por si difiere del nombre_cat)
            try:
                categoria = soup_detalle.find('ul', class_='breadcrumb').find_all('a')[-1].text.strip()
            except:
                categoria = nombre_cat

            # Intentamos extraer el stock desde el detalle (por si hay más info)
            try:
                stock = soup_detalle.find('p', class_='instock availability').text.strip()
            except:
                stock = ''

            # Guardamos toda la información del libro en un diccionario
            libro_info = {
                'Título': titulo,
                'Precio': precio,
                'Disponibilidad': disponibilidad,
                'Rating': rating,
                'Categoría': categoria,
                'Stock': stock,
                'URL': enlace_abs
            }

            # Agregamos el diccionario a la lista de libros
            libros_data.append(libro_info)

            # Imprimimos la información del libro en la terminal
            print(libro_info)

            # Pausamos un poco para no saturar el servidor
            time.sleep(0.1)

        # Pasamos a la siguiente página de la categoría
        page += 1

# Guardamos todos los libros recolectados en un archivo CSV
with open('libros_books_to_scrape.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=libros_data[0].keys())
    writer.writeheader()
    writer.writerows(libros_data)

print(f"Total de libros guardados: {len(libros_data)}")

ConnectTimeout: HTTPSConnectionPool(host='books.toscrape.com', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000017E5D194050>, 'Connection to books.toscrape.com timed out. (connect timeout=30)'))